# How to use AnySDK to make a dynamic toolkit

AnySDK allows you to wrap any SDK client that exposes various CRUD operations (get, update, create, delete) and turn them into LLM-enabled toolkits without having to write a custom toolkit.

There are two core aspects to AnySDK:
1. AnySDKWrapper -- wraps the SDK you pass in
2. CrudControls -- allows you to control the access an agent has to Create, Read, Update, Delete verbs.

As of writing, if you use Anthropic's Claude3 Haiku model in this notebook, it will cost less than USD 0.01 to complete.

First, we'll create a client SDK. Let's use AWS's `boto3` library. Ensure you have your credentials available or set them here. 

In [1]:
import os
import getpass

os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass(prompt="AWS Access Key ID: ")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass(prompt="AWS Secret Access Key: ")
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

Let's validate that your S3 client is able to make an API call. This `list_buckets` command should match how many buckets are in your account.

In [2]:
import boto3

s3 = boto3.client("s3")
buckets = s3.list_buckets()
print(len(buckets["Buckets"]))

9


Now, we can pass this client into AnySDKWrapper to create the toolkit. The callable functions within the SDK will become available as tools within the `anysdk.operations` object.

_NOTE_ If your SDK has many callable functions, your tool list could exceed your model's context length. Use CrudControls to limit the tools your Agent has access to.

To demonstrate this, we're limiting the `read_list` parameter to a single function, `list_buckets`.

In [3]:
from langchain_community.utilities.anysdk import AnySdkWrapper, CrudControls

client = {"client": s3}
crud_controls = CrudControls(read_list="list_buckets")

anysdk = AnySdkWrapper(client=client, crud_controls=crud_controls)

print(anysdk.operations)

[AnySDKTool(name='list_buckets', description=

.. note::

  

  This operation is not supported by directory buckets.

  

 

Returns a list of all buckets owned by the authenticated sender of the request. To use this operation, you must have the ``s3:ListAllMyBuckets`` permission.

 

For information about Amazon S3 buckets, see `Creating, configuring, and working with Amazon S3 buckets <https://docs.aws.amazon.com/AmazonS3/latest/userguide/creating-buckets-s3.html>`__.



See also: `AWS API Documentation <https://docs.aws.amazon.com/goto/WebAPI/s3-2006-03-01/ListBuckets>`_


**Request Syntax**

::

  response = client.list_buckets()
:rtype: dict
:returns: 
  
  **Response Syntax**

  
  ::

    {
        'Buckets': [
            {
                'Name': 'string',
                'CreationDate': datetime(2015, 1, 1)
            },
        ],
        'Owner': {
            'DisplayName': 'string',
            'ID': 'string'
        }
    }
    
  **Response Structure**

  

  - *(dict

Since `anysdk.operations` is a list of tools, we can pass that into an Agent upon initialization and use them.

In [4]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/structured-chat-agent")
prompt.pretty_print()

================================ System Message ================================

Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Final response to human"
}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation

==============

In [5]:
# Now we'll set the Anthropic API key
os.environ["ANTHROPIC_API_KEY"] = getpass.getpass(prompt="Anthropic API Key: ")

With your API key set, we'll be able to create the AgentExecutor.

In [6]:
from langchain_anthropic import ChatAnthropic
from langchain.agents import Agent, create_structured_chat_agent, AgentExecutor

llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)

agent = create_structured_chat_agent(llm, anysdk.operations, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=anysdk.operations,
    verbose=True,
    handle_parsing_errors=True,
)

Now we can use this agent to perform functions for us using the configured AWS `boto3` client! Let's ask the Agent how many buckets we have.

In [7]:
agent_executor.invoke(
    {
        "input": "How many S3 buckets do I have? Pass an empty string as the 'action_input'."
    }
)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "list_buckets",
  "action_input": ""
}
```
{"ResponseMetadata": {"RequestId": "B5FY500D641SZKPG", "HostId": "idqooATx+jCrS3C5IS94Ks9oRvCaQBScEwY7CQskQuGimTF5LWrLbeVLe3vYECr3cKdoPKVhH+4=", "HTTPStatusCode": 200, "HTTPHeaders": {"x-amz-id-2": "idqooATx+jCrS3C5IS94Ks9oRvCaQBScEwY7CQskQuGimTF5LWrLbeVLe3vYECr3cKdoPKVhH+4=", "x-amz-request-id": "B5FY500D641SZKPG", "date": "Mon, 13 May 2024 20:00:45 GMT", "content-type": "application/xml", "transfer-encoding": "chunked", "server": "AmazonS3"}, "RetryAttempts": 0}, "Buckets": [{"Name": "amplify-carboncounting-dev-100223-deployment", "CreationDate": "2019-10-26 14:02:32+00:00"}, {"Name": "casestudypro-api-dev-serverlessdeploymentbucket-1k1jafqbcjdzq", "CreationDate": "2022-02-19 19:12:41+00:00"}, {"Name": "casestudypro-api-prod-serverlessdeploymentbucket-12tvawyzsxx2l", "CreationDate": "2021-10-27 15:41:09+00:00"}, {"Name": "casestudypro-prod", "CreationDate": "2021-10-27 15:48:3

{'input': "How many S3 buckets do I have? Pass an empty string as the 'action_input'.",
 'output': 'You have 9 S3 buckets.'}

Now, let's have our Agent create a bucket. We'll need to modify the CrudControls to include the `create_bucket` function from the `boto3` SDK.

We'll also need to enable the create actions as well by setting `create` to `True`.

_WARNING_ This will create a bucket in your account! (Buckets are free to create, so long as you don't store data in them.)

In [8]:
from langchain_community.utilities.anysdk import AnySdkWrapper, CrudControls

client = {"client": s3}
crud_controls = CrudControls(
    read_list="list_buckets", create=True, create_list="create_bucket"
)

anysdk = AnySdkWrapper(client=client, crud_controls=crud_controls)

print(anysdk.operations)

[AnySDKTool(name='create_bucket', description=

.. note::

  

  This action creates an Amazon S3 bucket. To create an Amazon S3 on Outposts bucket, see `CreateBucket <https://docs.aws.amazon.com/AmazonS3/latest/API/API_control_CreateBucket.html>`__.

  

 

Creates a new S3 bucket. To create a bucket, you must set up Amazon S3 and have a valid Amazon Web Services Access Key ID to authenticate requests. Anonymous requests are never allowed to create buckets. By creating the bucket, you become the bucket owner.

 

There are two types of buckets: general purpose buckets and directory buckets. For more information about these bucket types, see `Creating, configuring, and working with Amazon S3 buckets <https://docs.aws.amazon.com/AmazonS3/latest/userguide/creating-buckets-s3.html>`__ in the *Amazon S3 User Guide*.

 

.. note::

  

  
  * **General purpose buckets** - If you send your ``CreateBucket`` request to the ``s3.amazonaws.com`` global endpoint, the request goes to the ``us-east

More complicated tools often require more complex logic which requires more capable models. To create a bucket, we'll use the Sonnet model available from Anthropic.

In [9]:
from langchain_anthropic import ChatAnthropic
from langchain.agents import Agent, create_structured_chat_agent, AgentExecutor

llm = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0)

agent = create_structured_chat_agent(llm, anysdk.operations, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=anysdk.operations,
    verbose=True,
    handle_parsing_errors=True,
)

In [10]:
import random

rand = str(random.randint(0, 10000000))
agent_executor.invoke(
    {"input": f"Create a bucket named 'my-test-bucket-{rand}' in my account."}
)



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "create_bucket",
  "action_input": {
    "Bucket": "my-test-bucket-6023880"
  }
}
```


TypeError: AnySDKTool._run() got an unexpected keyword argument 'Bucket'

To clean up the bucket you just created, run the below code block. Of course, you could also clean up with the Agent by modifying the CrudControls accordingly. 

In [30]:
s3.delete_bucket(Bucket=f"my-test-bucket-{rand}")

NoSuchBucket: An error occurred (NoSuchBucket) when calling the DeleteBucket operation: The specified bucket does not exist